In [1]:
# Import các thư viện cần thiết
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import (DecisionTreeClassifier,
                                       RandomForestClassifier, 
                                       GBTClassifier,
                                       MultilayerPerceptronClassifier)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from hdfs import InsecureClient

In [2]:
# Kết nối với HDFS và tạo Spark Session
client = InsecureClient('http://localhost:9870', user='thanhtin')
spark = (SparkSession.builder.appName("trainmodel_phanlop")
         .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000")
         .getOrCreate())

In [3]:
# Đường dẫn file HDFS
train_path = "hdfs://localhost:9000/thanhtin/train-airline.csv"
test_path = "hdfs://localhost:9000/thanhtin/test-airline.csv"

# Đọc file train và test từ HDFS vào DataFrame
train_df = spark.read.csv(train_path, header=True, inferSchema=True)
test_df = spark.read.csv(test_path, header=True, inferSchema=True)

# Loại bỏ các giá trị null nếu có
train_df = train_df.na.drop()
test_df = test_df.na.drop()

# Hiển thị một vài dòng dữ liệu để kiểm tra
train_df.show(5)
test_df.show(5)


+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|    id|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+-------------

In [4]:
# Mã hóa các cột chuỗi thành dạng số bằng StringIndexer
indexers = [
    StringIndexer(inputCol=column, outputCol=column + "_index").fit(train_df)
    for column in ["Gender", "Customer Type", "Type of Travel", "Class"]
]

for indexer in indexers:
    train_df = indexer.transform(train_df)
    test_df = indexer.transform(test_df)

# Cập nhật danh sách các cột đặc trưng sau khi thêm các cột đã được mã hóa
feature_columns = ["Age", "Flight Distance", "Inflight wifi service", "Departure/Arrival time convenient",
                   "Ease of Online booking", "Gate location", "Food and drink", "Online boarding", 
                   "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", 
                   "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", 
                   "Departure Delay in Minutes", "Arrival Delay in Minutes", 
                   "Gender_index", "Customer Type_index", "Type of Travel_index", "Class_index"]

# Tạo cột 'features' với VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

# Chuyển cột 'satisfaction' thành nhãn số (label) chỉ trên tập train, sau đó dùng lại cho test
satisfaction_indexer = StringIndexer(inputCol="satisfaction", outputCol="label").fit(train_df)
train_df = satisfaction_indexer.transform(train_df)
test_df = satisfaction_indexer.transform(test_df)

# Chia dữ liệu train thành tập train và tập validation
train_data, val_data = train_df.randomSplit([0.8, 0.2], seed=42)

# Hiển thị kết quả sau khi tiền xử lý
train_df.select("features", "label").show(5)


+--------------------+-----+
|            features|label|
+--------------------+-----+
|[13.0,460.0,3.0,4...|  0.0|
|[25.0,235.0,3.0,2...|  0.0|
|[26.0,1142.0,2.0,...|  1.0|
|[25.0,562.0,2.0,5...|  0.0|
|[61.0,214.0,3.0,3...|  1.0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Khởi tạo và huấn luyện các mô hình
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")
dt_model = dt.fit(train_data)

# rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=150, maxDepth=15)
# rf_model = rf.fit(train_data)

# gbt = GBTClassifier(featuresCol="features", labelCol="label", maxIter=15)
# gbt_model = gbt.fit(train_data)



# Dự đoán trên tập validation
predictions = dt_model.transform(val_data)
predictions.select("features", "label", "prediction").show(5)
# rf_predictions = rf_model.transform(val_data)
# rf_predictions.select("features", "label", "prediction").show(5)
# gbt_predictions = gbt_model.transform(val_data)
# gbt_predictions.select("features", "label", "prediction").show(5)


In [6]:
# Đánh giá mô hình bằng độ chính xác
# Đánh giá độ chính xác của từng mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(predictions)
# rf_accuracy = evaluator.evaluate(rf_predictions)
# gbt_accuracy = evaluator.evaluate(gbt_predictions)

# print(f"Decision Tree Accuracy: {dt_accuracy}")
print(f"Random Forest Accuracy: {dt_accuracy}")
# print(f"Gradient Boosted Tree Accuracy: {gbt_accuracy}")

Random Forest Accuracy: 0.9466420449023228


In [14]:
# Dự đoán trên tập test (sử dụng mô hình Random Forest)
test_predictions = rf_model.transform(test_df)
test_predictions.select("features","label", "prediction").show(10)

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[52.0,160.0,5.0,4...|  1.0|       1.0|
|[36.0,2863.0,1.0,...|  1.0|       1.0|
|[20.0,192.0,2.0,0...|  0.0|       0.0|
|[44.0,3377.0,0.0,...|  1.0|       1.0|
|[49.0,1182.0,2.0,...|  1.0|       0.0|
|[16.0,311.0,3.0,3...|  1.0|       1.0|
|[77.0,3987.0,5.0,...|  1.0|       1.0|
|[43.0,2556.0,2.0,...|  1.0|       1.0|
|[47.0,556.0,5.0,2...|  1.0|       1.0|
|[46.0,1744.0,2.0,...|  1.0|       1.0|
+--------------------+-----+----------+
only showing top 10 rows



In [13]:
# Lưu mô hình vào HDFS thư mục /thanhtin
rf_model.save("hdfs://localhost:9000/thanhtin/rf_model")


In [1]:
spark.stop()

NameError: name 'spark' is not defined